In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random as rd
import time

In [ ]:
class Vehicle(self):
    '''General vehicle class'''
    
    def __init__(self, pos=(-1,0), vel=(.1,0), risky=False):
        '''Defines the vehicle'''
        self.position = pos
        self.velocity = vel
        self.risky = risky

    # updates position based on velocity
    def move(self):
        self.position = (self.position[0]+self.velocity[0], self.position[1]+self.velocity[1])
    
    # sets position of vehicle
    def set_pos(self, pos):
        self.position = pos
        
    # gets position of vehicle
    def get_pos(self):
        return self.position
    
    # sets velocity of vehicle
    def set_vel(self, vel):
        self.velocity = vel
        
    # gets velocity of vehicle
    def get_vel(self):
        return self.velocity

In [ ]:
class Intersection():
    '''General intersection class'''
    
    def __init__(self):
        '''Defines the intersection'''
        self.vehicles = []
    
    # adds one vehicle to the intersection
    def add_vehicle(self, vehicle):
        self.vehicles.append(vehicle)
    
    # graphs a static visual of the intersection, roads, and vehicles
    def show(self):
        pass
    
    # advances the vehicles generally for all intersections
    def timestep(self):
        pass
    
    # loops over timesteps while displaying snapshots
    def animate(self):
        pass


In [ ]:
class Stopsign(Intersection):    
    '''Subclass of Intersection that carries behavior of the 4-way stop'''
    
    def __init__(self):
        super().__init__()
    
    # includes unique behavior of 4-way stop
    def timestep(self):
        pass

In [ ]:
class Stoplight(Intersection):    
    '''Subclass of Intersection that carries behavior of the stoplight'''
    
    def __init__(self):
        super().__init__()
    
    # includes unique behavior of stoplight
    def timestep(self):
        pass